# Свёрточные сети. Алгоритм Adam

Для агрегатора магазинов одежды нужно сделать классификатор на 10 классов. Каждый класс будет отвечать за определённую категорию товаров и отражён на сайте агрегатора:  

Label	Description  
0	    T-shirt/top  
1	    Trouser  
2	    Pullover  
3	    Dress  
4	    Coat  
5	    Sandal  
6	    Shirt  
7	    Sneaker  
8	    Bag  
9	    Ankle boot  

Мы будем работать с изображениями, потому что описания товаров могут содержать некорректную информацию, их может быть слишком много и причём на разных языках.  
На входе модель будет получать чёрно-белую фотографию товара.  

Задача построить и обучить нейронную сеть на наборе данных с предметами одежды.  
При работе с нейронными сетями требуются большие вычислительные мощности. Поэтому модель будет запускаться на сервере.  

**Описание данных**  
Данные находятся в датасете `fashion-mnist`.  Размер обучающей выборки: (60000, 28, 28), размер тестовой выборки: (10000, 28, 28).

In [ ]:
# Необходимые импорты
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.layers import Conv2D, Flatten, Dense, AvgPool2D, MaxPool2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np


# Загружаем обучающую выборку
def load_train(path):
    # загрузка признаков тренировочной выборки
    features_train = np.load(path + 'train_features.npy')
    # заргузка целевого признака тренировочной выборки
    target_train = np.load(path + 'train_target.npy')
    # приведение изображений в диапазон [0, 1]
    features_train = features_train.reshape(-1, 28, 28, 1) / 255.
    # результат: возвращение загруженных данных тестовой выборки (тестовые данные сервер грузит сам)
    return features_train, target_train

# Создаём модель
def create_model(input_shape):
    # инициализация модели
    model = Sequential()
    # добавление слоёв в модель
    # свёрточный слой с 6 фильтрами размером 3*3 с функцией активации `ReLU`
    model.add(Conv2D(6, (3, 3), padding='same', activation='relu', input_shape=(28, 28, 1)))
    # Average Pooling размером 2*2
    model.add(AvgPool2D(pool_size=(2, 2)))
    # разглаживание
    model.add(Flatten())
    # полносвязный слой с 10 нейронами, функция активации `SoftMax`
    model.add(Dense(units=10, activation='softmax'))
    # подключаем класс алгоритма с автоматическим подбором параметров для нейронов
    optimizer = Adam(lr=0.01)
    # готовим модель к обучению
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['acc'])
    # результат: возвращение настроенной модели
    return model

# Обучаем модель
def train_model(model, train_data, test_data, batch_size=38, epochs=10,
               steps_per_epoch=None, validation_steps=None):
    # назначение тренировочных и тестовых данных
    features_train, target_train = train_data
    features_test, target_test = test_data
    # обучение модели
    model.fit(features_train, target_train, 
              validation_data=(features_test, target_test),
              batch_size=batch_size, epochs=epochs,
              steps_per_epoch=steps_per_epoch,
              validation_steps=validation_steps,
              verbose=2, shuffle=True)
    # результат: возвращение обученной модели
    return model

**Вывод.** Модель обучена. Метрика `accuracy` на тренировочной выборке 0.9066, на валидационной 0.8878, на тестовой 0.8878. Модель показала отличный результат.